In [0]:
USE magnusp_catalog.training;

CREATE OR REFRESH STREAMING TABLE CarPartLifespan
SCHEDULE REFRESH EVERY 8 WEEK
COMMENT 'This table is used to track the lifespan of car parts'
TBLPROPERTIES ('pipelines.channel' = 'preview')
AS
SELECT 
  Component
, Description
, `Typical Material` as Material
, `Estimated Lifespan` as lifespan
from STREAM read_files (
  '/Volumes/magnusp_catalog/training/source/*.csv'
, format => 'csv'
, header => true
, mode => 'FAILFAST'
)
GROUP BY ALL
;

In [0]:
USE magnusp_catalog.training;
CREATE OR REFRESH STREAMING TABLE CarPartLifespan3 (
  Component STRING,
  Description STRING MASK magnusp_catalog.row_col_governance.general_mask_string using columns (
    'magnusp_catalog', 'training', 'carpartlifespan3','Description'
    ),
  Material STRING MASK magnusp_catalog.row_col_governance.general_mask_string using columns (
    'magnusp_catalog', 'training', 'carpartlifespan3','Material'
    ),
  lifespan STRING,
  CONSTRAINT CarPartLifespan3_pk PRIMARY KEY (Component, Description,Material) RELY
)
SCHEDULE REFRESH EVERY 8 WEEK
COMMENT 'This table is used to track the lifespan of car parts'
TBLPROPERTIES ('pipelines.channel' = 'preview')
AS
SELECT 
  Component
, Description
, `Typical Material` as Material
, `Estimated Lifespan` as lifespan
from STREAM read_files (
  '/Volumes/magnusp_catalog/training/source/*.csv'
, format => 'csv'
, header => true
, mode => 'FAILFAST'
)
GROUP BY ALL
;

In [0]:
CREATE OR REPLACE MATERIALIZED VIEW magnusp_catalog.training.Lifespan_And_Components
AS
WITH T1 AS (
  SELECT
    CASE  
      WHEN (lifespan) like '% years' THEN 'YEARS'
      WHEN (lifespan) like '% months' THEN 'MONTHS'
      WHEN lower(lifespan) like '% miles%' THEN 'MILES'
      WHEN lower(lifespan) like '% oil change%' THEN 'OIL'
      WHEN lower(lifespan) like '%lifetime%' THEN 'VEHICLE'
      ELSE 'OTHER'
      END as unit
  , if (
        contains(lower(lifespan), "-"), 
          cast(replace(trim(split(lower(lifespan),"-")[0]), ",", "") as INT), 
        if(
            contains(lower(lifespan),"miles") = true OR contains(lower(lifespan),"years") = true, 
              cast(replace(trim(split(lower(lifespan)," ")[0]), ",", "") as INT),
            NULL
        )) as lifespan_min
  ,  if (
          contains(lower(lifespan), "-"), 
            cast(replace(split(trim(split(lower(lifespan),"-")[1]), " ")[0], ",", "") as int), NULL
    ) as lifespan_max
  , array_sort(collect_set(lower(component))) as components 
  FROM 
    magnusp_catalog.training.CarPartLifespan3
  GROUP BY 1,2,3
  ORDER BY 1,2
)
  SELECT 
    * except(components)
  , posexplode(components) as (component_pos, component) 
  FROM T1
;

In [0]:
USE magnusp_catalog.training;
CREATE OR REPLACE MATERIALIZED VIEW magnusp_catalog.training.Lifespan_And_Components_V2
AS
WITH T1 AS (
  SELECT
    replace (
      replace ( 
        replace (
          lifespan, 
          " ", 
          "<SPC>"
        ), 
        "-", 
        "<DASH>"
      ),
      ",", 
      "<COMMA>"
    ) as lifespan_split_space
  , component
  FROM carpartlifespan3
),
T2 (
  SELECT 
    filter (
      flatten (
        transform (
          split(lifespan_split_space, "<SPC>"),  -- INPUT ARRAY
            x -> split ( -- FOR EACH ELEMENT IN ARRAY SPLIT
              replace (
                x, -- FIELD
                "<COMMA>", -- WHAT TO REPLACE
                "" -- REPLACE WITH EMPTY
              ), 
              "<DASH>" -- FIELD TO SPLIT ON
            )
        )
      ),  
      x-> lower(x) NOT IN (
        "", 
        "of",
        "the", 
        "(belt)", 
        "with", 
        "each", 
        "change", 
        "lifetime", 
        "replace"
      ) 
    )  as lifespan_split_space 
  , component
  FROM t1
), T3 (
  SELECT 
    upper(slice(reverse(lifespan_split_space),1, 1)[0]) as unit
  , reverse(slice(reverse(lifespan_split_space),2, size(lifespan_split_space)-1)) as rest
  , array_sort(collect_set(lower(component))) as components
  FROM T2
  GROUP BY 1,2
) , T4 (
  SELECT 
    unit
  , CAST(rest[0] as INT) as lifespan_min
  , CAST(rest[1] AS INT) as lifespan_max 
  , posexplode(components) (component_pos, component)
  FROM T3
)
SELECT * FROM T4

In [0]:
SELECT * FROM magnusp_catalog.training.Lifespan_And_Components_V2
ORDER BY 1,2,3,4;

##Functions introduced:
* posexplode
* array_sort
* reverse
* transform
* split
* slice
* size
* contains
* filter
* collect_set
* flatten
* cast

Summary:
Databricks has lots of functions not available in other languages. The reason for this is since we are not working with well prepared sets, we support nested structures, and can process in structure very efficiently

In [0]:
%python
from pyspark.sql import Row
from pyspark.sql.functions import posexplode

df = spark.createDataFrame([Row(a=1, intlist=[1,2,3], mapfield={"a": "b"})])
display(df.select(posexplode(df.intlist)))


In [0]:
%python
from pyspark.sql import Row
from pyspark.sql.functions import posexplode_outer

df = spark.createDataFrame(
    [(1, ["foo", "bar"], {"x": 1.0}), (2, [], {}), (3, None, None)],
    ("id", "an_array", "a_map")
)
display(df.select("id", "an_array", posexplode_outer("a_map")))









In [0]:
%python
from pyspark.sql.functions import lit, array_sort, length, when
df = spark.createDataFrame([([2, 1, None, 3],),([1],),([],)], ['data'])
df.select(array_sort(df.data).alias('r')).collect()

df = spark.createDataFrame([(["foo", "foobar", None, "bar"],),(["foo"],),([],)], ['data'])
df2 = df.select(array_sort(
    "data",
    lambda x, y: when(x.isNull() | y.isNull(), lit(0)).otherwise(length(y) - length(x))
).alias("r"))
display(df2)



In [0]:
%python
from pyspark.sql.functions import reverse
df = spark.createDataFrame([('Spark SQL',)], ['data'])
display(df.select(reverse(df.data).alias('s')));

df = spark.createDataFrame([([2, 1, 3,14],) ,([1,2],) ,([],)], ['data'])
display(df.select(reverse(df.data).alias('r')));


In [0]:
%python
from pyspark.sql.functions import transform

df = spark.createDataFrame([(1, [1, 2, 3, 4])], ("key", "values"))
display(df.select(transform("values", lambda x: x * 2).alias("doubled")))

# Alternatively you can define your own python function and pass in:

def alternate(x, i):
    return when(i % 2 == 0, x).otherwise(-x)

display(df.select(transform("values", alternate).alias("alternated")))






In [0]:
%python
from pyspark.sql.functions import split
df = spark.createDataFrame([('oneAtwoBthreeCfour',)], ['s',])
display(df.select(split(df.s, '[ABC]', 3).alias('s')))

display(df.select(split(df.s, '[ABC]', -1).alias('s')))
display(df.select(split(df.s, '[ABC]').alias('s')))



In [0]:
%python
from pyspark.sql.functions import slice

df = spark.createDataFrame([([1, 2, 3],), ([4, 5],)], ['x'])
df2 = df.select(slice(df.x, 2, 2).alias("sliced"))
display(df2)


In [0]:
%python
from pyspark.sql.functions import size

df = spark.createDataFrame([
  ([1, 2, 3],),
  ([1],),
  ([],),
  (None,)],
  ['data']
)
display(df.select(size(df.data)))


In [0]:
%python
from pyspark.sql.functions import contains
df = spark.createDataFrame([("Spark SQL", "Spark")], ['a', 'b'])
display(df.select(contains(df.a, df.b).alias('r')))


In [0]:
%python
from pyspark.sql.functions import filter, month, to_date

df = spark.createDataFrame(
    [(1, ["2018-09-20",  "2019-02-03", "2019-07-01", "2020-06-01", "2020-09-01"])],
    ("key", "values")
)
def after_second_quarter(x):
    return month(to_date(x)) > 6

df2 = df.select(
    filter("values", after_second_quarter).alias("after_second_quarter")
)

display(df2)
df2.printSchema()






In [0]:
%python
from pyspark.sql.functions import collect_set
df2 = spark.createDataFrame([(2,), (5,), (5,)], ('age',))
display(df2.agg(array_sort(collect_set('age')).alias('c')))


In [0]:
%python
from pyspark.sql.functions import flatten, array_compact
df = spark.createDataFrame([
  ([[1, 2, 3], [4, 5], [6]],), 
  ([None, [4, 5]],)], ['data'])
# Pre processing
display(df)

# flatten the array into one array
display(df.select(flatten(df.data).alias('r')))

# array_compact removes null values from array
display(df.select(flatten(array_compact(df.data).alias(''))))


In [0]:
%python
from pyspark.sql.types import StringType
df = spark.createDataFrame(
     [(2, "Alice"), (5, "Bob")], ["age", "name"])
display(df.select(df.age.cast("string").alias('ages')))

display(df.select(df.age.cast(StringType()).alias('ages')))


In [0]:
%python
df.explain(True)

In [0]:
%python
df.explain("extended")

In [0]:
%python
df = spark.sql("select * from magnusp_catalog.training.lifespan_and_components_v2")
df.explain(True)


In [0]:
%python
df.explain("cost")

In [0]:
%python
df.explain("codegen")

In [0]:
%python
x = spark.read.format("delta").load("/Volumes/magnusp_catalog/database1/sources/helly_test")

In [0]:
%python
from typing import Iterator
import pandas as pd
from pyspark.sql.functions import col,lit

def renormalize(itr) -> Iterator[pd.DataFrame]:
    for df in itr:
        x = df.explode("attribs")
        yield pd.DataFrame( {'name': x['_name'], 'default': x['_default']})
            
df = x.select(col('xs:complexType.xs:attribute').alias("attribs"))
#expected_schema = 'name string, default string'
#df = df.mapInPandas(renormalize, expected_schema)

pdf = df.toPandas().transpose()


In [0]:
%python
x = spark.read.table("magnusp_catalog.training.lifespan_and_components_v2").alias("x") 
y = spark.read.table("magnusp_catalog.training.lifespan_and_components").alias("y")

x2 = x.subtract(y)
display(x2)
x3 = y.subtract(x)
display(x3)

In [0]:
%python
display(x2)

In [0]:
%python
x = spark.read.table("magnusp_catalog.training.lifespan_and_components_v2").alias("x") 
y = spark.read.table("magnusp_catalog.training.lifespan_and_components").alias("y")

x2 = x.join(y, ["unit", "component", "component_pos"], "left_anti")
display(x2)

x3 = x.join(y, ["unit", "component", "component_pos"], "left")
display(x3)

x4 = x.join(y, ["unit", "component", "component_pos"], "right")
display(x4)

x5 = x.join(y, ["unit", "component", "component_pos"], "FULL")
display(x5)
x5.printSchema()


In [0]:

WITH T1 (
SELECT T2,T3,T4POS, T4
  LATERAL VIEW       EXPLODE(sequence(1, 100)) as T2
  LATERAL VIEW OUTER EXPLODE(shuffle(sequence(1000,1010))) as T3
  LATERAL VIEW       POSEXPLODE(shuffle(sequence(1000,1010))) as T4POS,T4
ORDER BY 1,2
)
SELECT * FROM T1 TABLESAMPLE(10 PERCENT)


In [0]:
select unit,count(*) cnt from magnusp_catalog.training.lifespan_and_components_v2
GROUP BY 1
ORDER BY 2 DESC

In [0]:
%python
x = spark.read.table("magnusp_catalog.training.Lifespan_And_Components_V2")
y = x.repartitionByRange(5,'unit')
y.write.format("delta").mode("overwrite").save("/Volumes/magnusp_catalog/database1/sources/partitioned_set")

In [0]:
%python
import pandas as pd

y = spark.read.format("delta").load("/Volumes/magnusp_catalog/database1/sources/partitioned_set").select("*","_metadata.file_name")
y1 = y.groupBy("file_name").count().orderBy("file_name")
display(y1)
